# Evaluation for the Synthetic Pendulum Experiments - Posed Approach
This notebook can be used to load and analyze the results generated by running `training_syntheticPendulum_posed.py`. The notebook is meant to evaluate multiple experiments together and compute the average Performances over all experiments. (E.g. the average performance over all 9 stonewall/woodwall/wallclock scenes, as reported in the papaer)

The path to the the folder containing the experiments needs to be specified under `path_experiment`. The folder set here needs to contain the subfolders with the experiments, where those subfolders need to contain `ckpt.pth` and `./hydra/`.

In [ ]:
%cd ..

In [ ]:
import os
import torch
import yaml
from models.sceneRepresentation import Scene
from dataset.dataset import DynamicPixelDataset
import matplotlib.pyplot as plt
from torchvision import utils
from util.util import compute_psnr, compute_iou

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

## Quantitative evaluation

In [ ]:
# Set the path to the experiments to analyze.
# The folder specified here needs to contain the subfolders which contain `ckpt.pth` and `./hydra/` 
path_experiments = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-24',
    'synthetic',
    'posed',
    'stonewall'
)

In [ ]:
psnrs = []
ious = []
path_file = os.path.join(path_experiments, 'results.txt')

# Delete previous result files
if os.path.exists(path_file):
    os.remove(path_file)

# Get all the experiments
for path_experiment in os.scandir(path_experiments):
    # Load Config
    path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
    with open(path_conf) as f:
        cfg = yaml.safe_load(f)

    print("Doing: ", cfg['data']['path_data'])

    # Load Model
    model = Scene(**cfg['scene']['background'])
    model.add_posedLocal(
        t_poses=torch.zeros(cfg['data']['max_samples']),
         **cfg['scene']['local_representation']
    )

    path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
    model.load_state_dict(torch.load(path_ckpt))

    model.to(device)

    # Load Data
    path_data = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])
    data = DynamicPixelDataset(
        path_data,
        skip_timesteps=cfg['data']['skip_timesteps'],
        start_index=cfg['data']['start_idx_test'],
        max_samples=cfg['data']['max_samples_eval']
    )
    H, W = data.get_image_dim()

    # Compute PSNR and IoU
    tspan = data.parameters["tspan"].to(device)
    model.update_trafo(tspan)
    output = model.render_image(W, H)
    psnr = compute_psnr(output['Image'].cpu(), data.get_full_images())
    iou = compute_iou(output['Mask'].cpu(), data.get_full_mask())
    psnrs.append(psnr)
    ious.append(iou)

    # Write summary file
    with open(path_file, 'a') as f:
        f.write(f"{cfg['data']['path_data']}\n")
        f.write(f"{path_experiment.path}\n")
        f.write(f"PSNR: {psnr}\n")
        f.write(f"IoU: {iou}\n")
        f.write("=============================\n\n")

    print(f"PSNR: {psnr}, IoU: {iou}")

    print("Done")
    print("====================================================")

avg_psnr = torch.mean(torch.tensor(psnrs))
avg_iou = torch.mean(torch.tensor(ious))
print("Results:")
print(f"Avg PSNR: {avg_psnr}, Avg IoU: {avg_iou}")

# Write to results file
with open(path_file, 'a') as f:
    f.write("\nLatex\n")
    f.write(f"{avg_psnr:.2f} & {avg_iou:.2f} & -")

## Render Images
The following code creates images for a specific experiment. The folder containing `ckpt.pth` and `./hydra/` for this experiment needs to be specified under `path_experiment`.

In [ ]:
path_experiment = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-24',
    'synthetic',
    'posed',
    'stonewall',
    '18-01-13_seq1'
)

path_folder = os.path.join(path_experiment, 'renderings')
if not os.path.isdir(path_folder):
    os.makedirs(path_folder)

# Load Config
path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
with open(path_conf) as f:
    cfg = yaml.safe_load(f)

# Load Model
model = Scene(**cfg['scene']['background'])
model.add_posedLocal(
    t_poses=torch.zeros(cfg['data']['max_samples']),
        **cfg['scene']['local_representation']
)

path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
model.load_state_dict(torch.load(path_ckpt))

model.to(device)
print("Model loaded")

In [ ]:
# Train images
# Load Data
path_data = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])
data = DynamicPixelDataset(
    path_data,
    skip_timesteps=cfg['data']['skip_timesteps'],
    max_samples=cfg['data']['max_samples']
)
H, W = data.get_image_dim()
print("Data loaded")

# Render
tspan = data.parameters["tspan"].to(device)
print(tspan)
model.update_trafo(tspan)
output = model.render_image(W, H)
ims = output["Image"].cpu()
masks = output['Mask'].cpu()

plt.imshow(ims[0])